The goal is to take a single DNA strand and explore the local space to see how large it is. This will help us understand the landscape of the functions we're looking for.

I will remove the restriction from the ALMresp and VMresp criteria (that it should end early) and establish a maximal score for Jon's weights.

Starting at Jon's weight, I will vary the weights systematically and randomly, and see what the landscape is, forming equivalued contours. I will plot the contours for each weight independently (weight on x-axis, score on y-axis).
I should plot bands of equal score, and see how the bands change as we move away from Jon's weights.

Optional: I will also plot the contours for each weight pair (weight1 on x-axis, weight2 on y-axis).

Optional:I will also plot the contours for each weight triplet (weight1 on x-axis, weight2 on y-axis, weight3 on z-axis).

I want to find out which directions of variance (weights) have the most impact on the score, and which are the most flexible. I will quantify the size of this 48-dimensional space and compare it to the overall size of the full 1000^48 space. 

In [3]:
# imports  
import sys, os

# # Add the src directory to sys.path
# src_path = os.path.join(os.path.dirname(__file__), 'src')
# sys.path.append(src_path)

import numpy as np
import pandas as pd
import shelve
import importlib
from src.neuron import *
from src.utils import *
from src.constants import * 
from src.network import *
from src.validation import *
from src.viz import *
from src.genetic_algorithm import *
from copy import copy
from datetime import datetime
from IPython.display import display, HTML
import sys, os
import time
from multiprocessing import Pool

In [2]:
# Establish score for Jon's weights

initial_dna = create_dna_string(new_jh_weights, ACTIVE_SYNAPSES)


### Settings ###
ga_set = "explore_B"
os.makedirs('./data', exist_ok=True)
save_path = f'./data/explore_{datetime.now().strftime("%Y-%m-%d_%H-%M-%S")}.pkl'

all_neurons = create_neurons()
splits, input_waves, alpha_array = create_experiment()
criteria_dict = define_criteria()
max_score = TMAX // BIN_SIZE * len(CRITERIA_NAMES)


sigma= GA_CONFIG[ga_set]['MUT_SIGMA'] * 10

curr_population = []
for _ in range(GA_CONFIG[ga_set]['POP_SIZE']):
    randomized_dna = np.round(np.random.normal(loc=initial_dna, scale=sigma)).astype(int)
    curr_population.append(randomized_dna)


save_dict = {}

for generation in range(GA_CONFIG[ga_set]['NUM_GENERATIONS']):
    print(f"Generation {generation}")
    population_results = []
    save_dict[f'{generation}'] = {}

    with Pool() as pool:
        args_list = [(dna, all_neurons, alpha_array, input_waves, criteria_dict, generation, max_score) 
                    for dna in curr_population]
        drone_results = pool.imap_unordered(drone_evaluate_dna, args_list)
        for curr_dna, total_score in drone_results:
            population_results.append(
                {'dna': curr_dna, 
                    'dna_score' : total_score
                    })

    # Quick save to temp dict (repository for all dna across all generations)   
    save_dict[f'{generation}'] = population_results

    curr_population = spawn_next_population(population_results, GA_CONFIG[ga_set], generation)

# Pickle run data 
with open(save_path,'ab') as f:
    pickle.dump(save_dict, f)


# test_dna=create_dna() # --> use simulated annealing



Generation 0
Gen 0 === DNA: [  34  -20    8  224  -92    6   -1   -5  -50   -5   -4   -6  -89   -6
    0   -7 -105   -2  -56   64   -6   -8    6   -6   91  320    1    1
    4    5    4  328   68   91    0  -11    4   -9   -1    8    2   -4
  -44    4    5    6   56   52   23]
    === Control: 249/500
    === Experimental: 377/500
    === Overall: 626(62.60%)

Gen 0 === DNA: [  38   -2   -3  207  -90    1    2   -2  -48   -2   -6   -4  -80  -20
    1    2 -105    0  -54   71   -5    1    6   -7   89  324   -5   -8
   -7    4    2  319   80   88   -1    1    5    0   -7   -3   -3   -3
  -54   -1   -3   -5   59   59   33]
    === Control: 247/500
    === Experimental: 408/500
    === Overall: 655(65.50%)

Gen 0 === DNA: [  38  -11    1  216  -95   -8  -10   -1  -53    4    5    3  -89   -8
    0   -7 -106    0  -52   72    7    0    4   -4   81  317    6   -4
    4   -4   -3  323   81   75   -2   -3    1    1    1    6    7    0
  -49    2    2    9   56   60   27]
    === Control: 249/5

KeyboardInterrupt: 

In [ ]:
#Establish